In [1]:
import pandas as pd
import time
from sklearn.model_selection import train_test_split
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from multiprocessing import Pool
from functools import partial
import math
from operator import itemgetter

In [12]:


def timeit(method):
    """
    Standard Python decorator that measures the execution time of a method;
    """
    def timed(*args, **kw):
        start = time.time()
        result = method(*args, **kw)
        end = time.time()
        
        print(f"{method.__name__}:  {(end - start):.2f} s")
        return result
    return timed

#############################
#############################

@timeit
def load_training_data(data_path: str, row_subset: float=1, train_split: float=0.7, shuffle: bool=False, seed=None):
    '''
    Load the training set and divide it into training and test splits.
    "LinkedID" is the value that we want to predict
    :param data_path: path to the dataset to load;
    :param row_subset: use only the specified fraction of rows in the dataset (value in (0, 1]);
    :param train_split: fraction of rows placed in the training set;
    :param shuffle: if True, shuffle the rows before splitting or subsetting the data;
    '''
    if row_subset <= 0 or row_subset > 1:
        row_subset = 1
    
    data = read_file(training_file, set_record_id_as_index=True)
    if shuffle:
        data = data.sample(frac=1, random_state=seed)
    # Obtain the specified subset of rows;
    data = data.iloc[:int(np.ceil(len(data) * row_subset))]
        
    X = data.drop(columns="linked_id")
    y = data["linked_id"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_split, shuffle=shuffle, random_state =seed)
    
    return X_train, X_test, y_train, y_test

#############################
#############################

@timeit
def clean_data(data: pd.DataFrame) -> pd.DataFrame:
    """
    Perform cleaning of the dataset, e.g. lowercase strings, fill missing values, etc...
    """
    cleaned_data = data.copy()
    for c in ["name", "type", "address", "phone", "email", "modification"]:
        cleaned_data[c] = cleaned_data[c].str.lower()
    
    return cleaned_data

#############################
#############################
    
@timeit
def index_data(data: list) -> list:
    """
    Manipulate the data to create indices that speed up the computation. 
    
    The dataframes to be indexed are passed with a list of pd.DataFrames.
    Be careful not to leak information from the train dataset to the test dataset!
    
    In this case, replace all strings with unique integer values,
    so that comparing rows for equality is much faster;
    """
    
    # Obtain the original ending of each dataframe.
    # Prepend a 0 to indicate the start of the first dataframe;
    lengths = [0] + [len(x) for x in data]
    lengths = np.cumsum(lengths)
    # Join together all the input data;
    concatenated_data = pd.concat(data)
        
    for c in ["email_domain","type", "address_country","phone_prefix","email_suffix","modification"]:
        concatenated_data[c] = pd.factorize(concatenated_data[c])[0]
    
    # Split the input data;
    indexed_data = []
    for i in range(len(lengths) - 1):
        indexed_data += [concatenated_data.iloc[lengths[i]:lengths[i + 1], :]]
    
    return indexed_data

#############################
#############################


    
def predict_record_fast(train: tuple, record: np.array, K: int=10) -> list:
    """
    Given a record and a training set, find the records in the training set that best match
    the input record. Predictions can be done using very simple unsupervised algorithms,
    as done in this example, or computed with some fancy ML model;
    """
    # In this simple model, look for the number of identical columns for each training record.
    # Skip the last column (type of modification);
    X_train,y_train=train[0],train[1]
    #identical_values_per_row = pd.Series(np.array(record[3:] == X_train[:,3:]).sum(axis=1))

    # Fuzzy match
    dataframecolumn0 = pd.DataFrame([record])
    dataframecolumn0.columns = ['Match0']
    
    compare0 = pd.DataFrame(X_train[:,0])
    compare0.columns = ['compare0']
    
    dataframecolumn0['Key0'] = 1
    compare0['Key0'] = 1
    combined_dataframe0 = dataframecolumn0.merge(compare0,on="Key0",how="left")
    
    identical_values_per_row = pd.Series(partial_match_vector_complessivo(combined_dataframe0['Match0'],combined_dataframe0['compare0']))
    '''
    # Fuzzy address match
    dataframecolumn1 = pd.DataFrame([record[1]])
    dataframecolumn1.columns = ['Match1']
    
    compare1 = pd.DataFrame(X_train[:,1])
    compare1.columns = ['compare1']
    
    dataframecolumn1['Key1'] = 1
    compare1['Key1'] = 1
    combined_dataframe1 = dataframecolumn1.merge(compare1,on="Key1",how="left")
    
    identical_values_per_row += partial_match_vector_address(combined_dataframe1['Match1'],combined_dataframe1['compare1'])

    # Fuzzy email_first_part match
    dataframecolumn2 = pd.DataFrame([record[3]])
    dataframecolumn2.columns = ['Match2']
    
    compare2 = pd.DataFrame(X_train[:,3])
    compare2.columns = ['compare2']
    
    dataframecolumn2['Key2'] = 1
    compare2['Key2'] = 1
    combined_dataframe2 = dataframecolumn2.merge(compare2,on="Key2",how="left")
    
    identical_values_per_row += partial_match_vector_email_first(combined_dataframe2['Match2'],combined_dataframe2['compare2'])

    #Fuzzy email_domain match
    dataframecolumn3 = pd.DataFrame([record[2]])
    dataframecolumn3.columns = ['Match3']
    
    compare3 = pd.DataFrame(X_train[:,2])
    compare3.columns = ['compare3']
    
    dataframecolumn3['Key3'] = 1
    compare3['Key3'] = 1
    combined_dataframe3 = dataframecolumn3.merge(compare3,on="Key3",how="left")
    
    identical_values_per_row += partial_match_vector_number_no_prefix(combined_dataframe3['Match3'],combined_dataframe3['compare3'])
    '''
    
    # Obtain the K rows with the most matches;
    best_matches = identical_values_per_row.sort_values(ascending=False)[:K]    
    # Retrieve the original record IDs from the training set;
    #print(best_matches)
    return list(zip(list(y_train.loc[best_matches.index.values]) ,list(best_matches)))


def predict_er_fast(X_train, y_train,info,soglia, K=10) -> dict:
#def predict_er_fast(train, X_test, K=10) -> dict:
    """
    Given a training dataset and a test dataset, obtain the top-K predictions 
    for each record in the test dataset;
    """

    # Store for each record the list of predictions;
    predictions = {}
    start = time.time()
    
    # Extract the matrix that represent the data.
    # Skip the last column;
    #X_train_matrix = X_train.values[:, :-1]
    
    # Also reindex the y column, as we lose the original index when doing a comparison using matrices;
    y_train_vector = y_train.reset_index(drop=True)
    #X_train=X_train.reset_index(drop=True)
    n_threads=18
    n_reduced=math.ceil(len(X_train)/float(n_threads))
    chunks=[X_train.iloc[i*n_reduced:(i+1)*n_reduced,1:].values for i in range(n_threads)]
    chunks_y=[y_train_vector.iloc[i*n_reduced:(i+1)*n_reduced].reset_index(drop=True) for i in range(n_threads)]
    print(chunks[0])
    # Compute a prediction for each record;
    i=0
    for i in range(1): #,X_test['email_domain'],X_test['type'],X_test['address_country'],X_test['phone_prefix'],X_test['email_suffix']):
        # Extract values from the current row;
        #row_i=(name,address,phone,mail)
        #row_i_vector = np.array(row_i)
        #print(row_i_vector)
        # Find the best matching record for the current record;
        pool=Pool(n_threads)
        input_data = tuple(zip(chunks,chunks_y))
        appoggio =[x[:] for x in pool.map(partial(predict_record_fast, record=info,K=10),input_data)]
        appoggio_2=[item for sublist  in appoggio for item in sublist]
        #print(appoggio_2)
        appoggio_3=sorted(appoggio_2,key=lambda x : float(x[1]),reverse=True)
        predictions[0]=[i for i in appoggio_3 if i[1]>=soglia][:K]
    return predictions


@timeit
def prediction_dict_to_df(predictions: dict) -> pd.DataFrame:
    # Turn the prediction dict into a series of tuples;
    results = []
    for query_id, pred_list in predictions.items():
        for p in pred_list:
            results += [[query_id, p]]
    return pd.DataFrame(results, columns=["queried_record_id", "predicted_record_id"])

@timeit
def prediction_dict_to_kaggle_df(predictions: dict) -> pd.DataFrame:
    # Turn the prediction dict into a series of tuples;
    results = []
    for query_id, pred_list in predictions.items():
        results += [[query_id, " ".join(pred_list)]]
    return pd.DataFrame(results, columns=["queried_record_id", "predicted_record_id"])

@timeit
def kaggle_sol_to_df(kaggle_df: pd.DataFrame) -> pd.DataFrame:
    kaggle_df_indexed = kaggle_df.set_index("queried_record_id")
    results = []
    for query_id, pred_list in kaggle_df_indexed.iterrows():
        results += [[query_id, pred] for pred in pred_list["predicted_record_id"].split(" ")]
    return pd.DataFrame(results, columns=["queried_record_id", "predicted_record_id"])

def read_file(path: str, set_record_id_as_index: bool=False) -> pd.DataFrame:
    return pd.read_csv(path, dtype=str, escapechar="\\", index_col="record_id" if set_record_id_as_index else None)


In [3]:
training_filez = "../panama-papers-polimi/data/entity-resolution_advanced-topics-training_data.csv"
train = read_file(training_filez, set_record_id_as_index=False)
#X_train_parra = train.drop(columns="linked_id")
y_train = train["record_id"]
#X_train, X_test, y_train, y_test = load_training_data(training_file, shuffle=True, row_subset=0.01, seed=42)

trainn=pd.read_csv('../panama-papers-polimi/data/panama_train_expanded_2.csv')
#trainn = read_file(training_file, set_record_id_as_index=True)

testt=pd.read_csv('../panama-papers-polimi/data/panama_train_expanded_2.csv')
#testt = read_file(testing_file, set_record_id_as_index=True)



In [4]:
X_train=trainn.copy()
X_test=testt.copy()

X_train=X_train[["record_id","name","address","email","phone"]] #"email_domain","type", "address_country","phone_prefix","email_suffix","modification"]]
X_test=X_test[["record_id","name","address","email","phone"]] #"email_domain","type" ,"address_country","phone_prefix","email_suffix","modification"]]


X_train.phone=X_train.phone.apply(str)
X_test.phone=X_test.phone.apply(str)

X_train=X_train.fillna('-1')
X_test=X_test.fillna('-1')

#Treat strings that are too short as missing values
X_train.name=X_train.name.apply(lambda x : '-1' if len(x)<=2 else x)
X_train.address=X_train.address.apply(lambda x : '-1' if len(x)<=2 else x)
X_train.email=X_train.email.apply(lambda x : '-1' if len(x)<=2 else x)
X_train.phone=X_train.phone.apply(lambda x : '-1' if len(x)<=2 else x)

X_test.name=X_test.name.apply(lambda x : '-1' if len(x)<=2 else x)
X_test.address=X_test.address.apply(lambda x : '-1' if len(x)<=2 else x)
X_test.email=X_test.email.apply(lambda x : '-1' if len(x)<=2 else x)
X_test.phone=X_test.phone.apply(lambda x : '-1' if len(x)<=2 else x)

X_train=X_train.fillna('-1')
X_test=X_test.fillna('-1')



#Create new merged features from name,address,phone,email

X_train_3=X_train.copy()
X_train_3.set_index('record_id',inplace=True)
X_train.name=X_train.name.apply(lambda x : '' if x=='-1' else x)
X_train.address=X_train.address.apply(lambda x : '' if x=='-1' else x)
X_train.email=X_train.email.apply(lambda x : '' if x=='-1' else x)
X_train.phone=X_train.phone.apply(lambda x : '' if x=='-1' else x)

X_test.name=X_test.name.apply(lambda x : '' if x=='-1' else x)
X_test.address=X_test.address.apply(lambda x : '' if x=='-1' else x)
X_test.email=X_test.email.apply(lambda x : '' if x=='-1' else x)
X_test.phone=X_test.phone.apply(lambda x : '' if x=='-1' else x)

X_train=X_train[['record_id','name','address','email','phone']]
X_train['complessivo']=X_train['name']+' '+X_train['address']+' '+X_train['email']+' '+X_train['phone']
#X_train.set_index('record_id',inplace=True)

X_test['complessivo']=X_test['name']+' '+X_test['address']+' '+X_test['email']+' '+X_test['phone']


X_train=X_train[['record_id','complessivo']]
X_test=X_test[['record_id','complessivo']]

In [11]:
def partial_match_name(x,y):
    return(fuzz.ratio(x,y)*0.092)
partial_match_vector_name = np.vectorize(partial_match_name)

def partial_match_address(x,y):
    return(fuzz.ratio(x,y)*0.002)
partial_match_vector_address = np.vectorize(partial_match_address)

def partial_match_email_first(x,y):
    return(fuzz.ratio(x,y)*0.002)
partial_match_vector_email_first = np.vectorize(partial_match_email_first)

def partial_match_number_no_prefix(x,y):
    return(fuzz.ratio(x,y)*0.004)
partial_match_vector_number_no_prefix = np.vectorize(partial_match_number_no_prefix)

def partial_match_complessivo(x,y):
    return(fuzz.token_set_ratio(x,y)*0.1)
partial_match_vector_complessivo = np.vectorize(partial_match_complessivo)



In [17]:
print('Welcome,please type the personal information of the person you need to check')

name = input('NAME :')
address = input('ADDRESS :')
phone= input('PHONE :')
mail=input('MAIL :')



Welcome,please type the personal information of the person you need to check
NAME :luca nardini
ADDRESS :
PHONE :
MAIL :


In [22]:
soglia = input('Please insert a threshold value between 0 and 10 :')
K=input('Lastly, please insert the number of matching records you would like to retrieve :')

Please insert a threshold value between 0 and 10 :0
Lastly, please insert the number of matching records you would like to retrieve :20


In [23]:
info=name+' '+address+' '+phone+' '+mail
info=np.array(info)
predictions = predict_er_fast(X_train, y_train,info,float(soglia),int(K))

[['ardia enterprisesmltd  imquires@zoho.vo 4102971003602']
 ['andre almeida blanco la rua pelotas 209 apartamento 72 bloco a en el municipio de sao paulo estado de sao paulo cep 0k012-000  ']
 ['moonta holdings ltd   65013464145']
 ...
 ['sterling mckay partners assets sa  sterlingmckaypartnersassetssa@outlook.de ']
 ['tinley holdings limited   446818347899']
 ['terrace property holdings ltd  inquires@icloud.de ']]


In [24]:
print('This is the list of the best matching candidates and their similarity scores you asked for : ')
print(predictions[0])
array=[]
for j in predictions[0]:
    if j[0]!='':
        array.append(j[0])



This is the list of the best matching candidates and their similarity scores you asked for : 
[('12223689-M0', 10.0), ('12223689-NV0', 10.0), ('12223689', 10.0), ('12223689-M1', 10.0), ('12223689-M2', 10.0), ('11002948-M2', 7.0), ('13009323-M2', 6.7), ('12112309-M0', 6.7), ('13009323-M1', 6.7), ('13009323-M0', 6.7), ('12150053-M1', 6.7), ('12112309-M1', 6.7), ('10158628-M0', 6.4), ('10158628', 6.4), ('10158628-T0', 6.4), ('10158628-T1', 6.4), ('12043922-T0', 6.2), ('10052529', 6.2), ('12223841-M0', 6.2), ('12089129', 6.2)]


In [25]:
print('And now you can check the personal information of the best matches : ')
print(X_train_3.loc[array,'name'])



And now you can check the personal information of the best matches : 
record_id
12223689-M0          luca nardini
12223689-NV0         luca nardini
12223689             luca nardini
12223689-M1          luca nardini
12223689-M2          luca nardini
11002948-M2           srini lanka
13009323-M2                lu nan
12112309-M0       luca gallinelli
13009323-M1                lu nan
13009323-M0                lu nan
12150053-M1          luca sandoli
12112309-M1       luca gallinelli
10158628-M0      luchy trading sa
10158628         luchy trading sa
10158628-T0      luchy trading sa
10158628-T1      luchy trading sa
12043922-T0        lua mariadmena
10052529           sardinali corp
12223841-M0        laurent nordin
12089129        lucila piacentini
Name: name, dtype: object
